In [2]:
from tensorflow.keras.models import load_model
import joblib

model = load_model("sepsis_lstm_model.keras")
scaler = joblib.load("scalers.pkl")

print("Model and scaler loaded successfully!")


Model and scaler loaded successfully!


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [8]:
import numpy as np

FEATURES = ['HR', 'SBP', 'RR', 'Temp', 'SpO2']

def predict_sepsis(patient_sequence):
    """
    Input must be shape (10,5):
    Features order: [HR, SBP, RR, Temp, SpO2]
    Returns sepsis probability + risk category
    """
    patient_sequence = np.array(patient_sequence)

    # Validate shape
    if patient_sequence.shape != (10, 5):
        raise ValueError("Input must be shape (10,5): 10 hours x 5 vital signs")

    # Scale each feature using its saved scaler
    patient_scaled = np.zeros_like(patient_sequence, dtype=float)
    for i, feature in enumerate(FEATURES):
        patient_scaled[:, i] = scaler[feature].transform(
            patient_sequence[:, i].reshape(-1, 1)
        ).flatten()

    # Reshape for LSTM input
    patient_input = patient_scaled.reshape(1, 10, 5)

    # Predict
    risk_prob = float(model.predict(patient_input)[0][0])

    # Categorize
    if risk_prob >= 0.8:
        risk_category = "High risk: Septic shock likely"
    elif risk_prob >= 0.5:
        risk_category = "Moderate risk: Sepsis likely"
    else:
        risk_category = "Low risk: Patient stable"

    return {
        "risk_probability": round(risk_prob, 2),
        "risk_category": risk_category,
        "features": FEATURES
    }


In [9]:
test_patient = [
    [95, 100, 22, 38.2, 97],
    [96, 99, 22, 38.3, 97],
    [97, 98, 23, 38.4, 96],
    [98, 97, 23, 38.5, 96],
    [99, 96, 24, 38.6, 95],
    [100, 95, 24, 38.7, 95],
    [101, 94, 25, 38.8, 94],
    [102, 93, 26, 38.9, 94],
    [103, 92, 27, 39.0, 93],
    [104, 91, 28, 39.1, 93]
]

predict_sepsis(test_patient)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step


{'risk_probability': 0.11,
 'risk_category': 'Low risk: Patient stable',
 'features': ['HR', 'SBP', 'RR', 'Temp', 'SpO2']}

In [10]:
# Bad patient vitals (severe sepsis / septic shock)
bad_patient_sequence = [
    [120, 70, 30, 39.5, 85],  # Hour 1
    [125, 65, 32, 39.8, 83],
    [130, 60, 34, 40.0, 82],
    [135, 55, 36, 40.2, 80],
    [140, 50, 38, 40.5, 78],
    [145, 45, 40, 40.8, 75],
    [150, 40, 42, 41.0, 73],
    [155, 35, 44, 41.2, 70],
    [160, 30, 46, 41.5, 68],
    [165, 25, 48, 42.0, 65]
]

result = predict_sepsis(bad_patient_sequence)
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
{'risk_probability': 0.72, 'risk_category': 'Moderate risk: Sepsis likely', 'features': ['HR', 'SBP', 'RR', 'Temp', 'SpO2']}


In [11]:
# Extreme septic shock vitals
shock_patient_sequence = [
    [140, 60, 35, 40.0, 82],  # Hour 1
    [145, 55, 37, 40.5, 80],
    [150, 50, 39, 41.0, 78],
    [155, 45, 41, 41.2, 75],
    [160, 40, 43, 41.5, 73],
    [165, 35, 45, 41.8, 70],
    [170, 30, 47, 42.0, 68],
    [175, 25, 49, 42.3, 65],
    [180, 20, 51, 42.5, 63],
    [185, 15, 55, 43.0, 60]
]

result = predict_sepsis(shock_patient_sequence)
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
{'risk_probability': 0.91, 'risk_category': 'High risk: Septic shock likely', 'features': ['HR', 'SBP', 'RR', 'Temp', 'SpO2']}
